# Overlay 

Visually stacking time

In [ ]:
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

sns.set(rc={'figure.figsize':(12,6)})
plt.style.use(['seaborn-whitegrid'])

import warnings
warnings.simplefilter('ignore')

In [ ]:
import utils

filename = 'data/measured_real_power.csv'
df = utils.read_csv(filename)

What is the total load as a function of time of day?

In [ ]:
total = df.sum(axis=1)
total.head()

In [ ]:
type(total)

In [ ]:
total = pd.DataFrame({'values': total})
total.head()

In [ ]:
total.plot()

Let's add some additional columns

In [ ]:
total['time'] = total.index.time
total['day'] = total.index.day
total.head()

Pivot with wide days

In [ ]:
pivot = total.pivot(index='time', columns='day', values='values')
pivot.head()

In [ ]:
ax  = pivot.plot(legend=False)

In [ ]:
mean = pivot.mean(axis=1)
mean.plot()

In [ ]:
mean = pivot.mean(axis=1)
ax = pivot.plot(legend=False, color=['grey' for x in pivot.columns], alpha=0.25)
ax = mean.plot(lw=2, color='red', ax=ax)

## Fill between

In [ ]:
import matplotlib.pyplot as plt

upper = mean + pivot.std(axis=1)
lower = mean - pivot.std(axis=1)

ax = mean.plot(lw=2, color='red')
_ = ax.fill_between(upper.index, upper.values, lower.values, color='grey', alpha=0.25)

Residuals: **broadcast** the mean

In [ ]:
pivot.subtract(mean, axis=0).head()

We want to flatten

In [ ]:
pivot.subtract(mean, axis=0).melt()['value'].plot(kind='hist', bins=30)

What if `std` isn't really a good description of what's happening?

## Distributions 

Let's resample a bit and look again.

In [ ]:
df = utils.read_csv("data/measured_real_power.csv")
df = df.resample("15min").sum().mean(axis=1)  # resample
stacked = df.reset_index().melt(id_vars='timestamp')
stacked['time'] = stacked.timestamp.map(lambda x: x.time())
stacked['day'] = stacked.timestamp.map(lambda x: x.day)
pivot = stacked.pivot_table(index='time', columns=['variable', 'day'], values='value')

In [ ]:
pivot.plot(legend=False, alpha=0.5)

In [ ]:
mean = pivot.mean(axis=1)

upper = mean + pivot.std(axis=1)
lower = mean - pivot.std(axis=1)

ax = mean.plot(lw=2, color='red')
_ = ax.fill_between(upper.index, upper.values, lower.values, color='grey', alpha=0.25)

In [ ]:
pivot.subtract(mean, axis=0).melt()['value'].plot(kind='hist', bins=20)

In [ ]:
quant_range = [0, 0.1, 0.25, 0.75, 0.9, 1.0]
quant = pivot.quantile(quant_range, axis=1).T
quant.head()

In [ ]:
pivot.mean(axis=1).plot(lw=2, color='red')
for r1,r2 in zip(quant_range, list(reversed(quant_range))):    
    plt.fill_between(quant.index, quant[r1].values, quant[r2].values, color='grey', alpha=0.1 )